In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc, math

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Preprocessing Test

In [10]:
X = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 12:00:00']}

In [2]:
def preprocess(X):

    weather_preproc = pd.read_csv('../raw_data/weather_preproc.csv', parse_dates=['timestamp'])

    building_preproc = pd.read_csv("../raw_data/building_preproc.csv")


    X = pd.DataFrame.from_dict(X)
    
    X['timestamp'] = pd.to_datetime(X.timestamp)
    row = X.merge(building_preproc.iloc[:,1:], on='building_id', how='left')

    row = row.merge(weather_preproc.iloc[:,1:], on=['site_id', 'timestamp'], how='left')

    beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
              (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

    for item in beaufort:
        row.loc[(row['wind_speed']>=item[1]) & (row['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

    row["timestamp"] = pd.to_datetime(row["timestamp"])

    def transform(df):
        df['hour'] = np.uint8(df['timestamp'].dt.hour)
        df['day'] = np.uint8(df['timestamp'].dt.day)
        df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
        df['month'] = np.uint8(df['timestamp'].dt.month)
        df['year'] = np.uint8(df['timestamp'].dt.year-1900)

        df['square_feet'] = np.log(df['square_feet'])

        return df

    row = transform(row)

    def encode_cyclic_feature(df, col, max_vals):
        df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
    #     df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
        del df[col]
        return df

    dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
    us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

    row['is_holiday'] = (row['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
    row['is_holiday'] = (row['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

    row.loc[(row['weekday']) == 5 | (row['weekday'] == 6) , 'is_holiday'] = 1

    row = row.drop(['timestamp'], axis=1)

    row = encode_cyclic_feature(row, 'weekday', 7)
    row = encode_cyclic_feature(row, 'hour', 24)
    row = encode_cyclic_feature(row, 'day', 31)
    row = encode_cyclic_feature(row, 'month', 12)

    row = row.loc[:,['site_id', 'building_id', 'primary_use', 'meter', 'wind_direction',
           'is_holiday', 'square_feet', 'year_built', 'air_temperature',
           'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'floor_count',
           'beaufort_scale', 'weekday_sin', 'day_sin', 'hour_sin', 'month_sin']]
    return row

In [11]:
X = preprocess(X)

In [3]:
def pred(X):
    model1 = lgb.Booster(model_file='model1.txt')
    model2 = lgb.Booster(model_file='model2.txt')
    y_pred = (model1.predict(X)+model2.predict(X))/2
    
    #to_return = {'meter_reading':float(y_pred)}

    return y_pred

## Monthly Prediction

In [101]:
dates = pd.date_range("2017-01-01 0:00", "2017-01-31 23:00", freq="1h")

input_values=pd.DataFrame(dates)
input_values['building_id'] = 100
input_values['meter'] = 4
input_values = input_values.iloc[:,[1,2,0]]
input_values = input_values.rename(columns={0:'timestamp'})

X = preprocess(input_values)


X_modified = X.copy()

X_modified.air_temperature += 3

X_modified.square_feet /=2

y_pred = pred(X)

y_pred_modified = pred(X_modified)

y =  pd.DataFrame({'Cons. (kwh)':y_pred,'Cons. (kwh) - Retrofit':y_pred_modified},index = input_values.timestamp)

y_json = y.to_json()

In [89]:
y_recovered = pd.read_json(y_json)
y_recovered

,Cons. (kwh),Cons. (kwh) - Retrofit
2017-01-01 00:00:00,4.623410,3.550319
2017-01-01 01:00:00,4.615687,3.521770
2017-01-01 02:00:00,4.665055,3.692216
2017-01-01 03:00:00,4.595930,3.643917
2017-01-01 04:00:00,4.647516,3.674677
...,...,...
2017-01-31 19:00:00,4.392853,3.441286
2017-01-31 20:00:00,4.656146,3.684061
2017-01-31 21:00:00,4.852225,3.774210
2017-01-31 22:00:00,4.769671,3.780903


In [51]:
import plotly.express as ptx

In [90]:
y_cumsum = y_recovered.resample('D').mean().cumsum()

In [91]:
ptx.line(y_cumsum)